# Installing and Importing Dependencies

In [ ]:
# MPS acceleration is available on MacOS 12.3+
#!pip3 install torch torchvision torchaudio

In [ ]:
#!pip3 install transformers requests beautifulsoup4 numpy pandas

In [ ]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
from bs4 import BeautifulSoup
import re
import requests
import torch
import pandas as pd
import numpy as np

# instantiating the model

In [16]:
tokenization_tool = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [42]:
tokens = tokenization_tool.encode('I ll become a data scientist or machine learning engineer soon ', return_tensors='pt')
result = model(tokens)

In [43]:
int(torch.argmax(result.logits))+1

5

### Now some scraping : i ll scrape data(reviews about Dishoom restaurant in London) from yelp.com

In [54]:
req = requests.get("https://www.yelp.com/biz/dishoom-london?osq=Restaurants")
soup = BeautifulSoup(req.text,"html.parser")
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [61]:
reviews[0]

"This was a total hit. We made a reservation for lunch which I highly recommend because this place had a line out the door despite not even being open when we arrived. It's so cutely decorated. Tons of nods to the original Cafe Britannia in Bombay and overall seems to serve as a beautiful homage to India/Bombay. The food itself is incredible. Chili cheese toast was authentic. Pau bhaji and chicken were sumptuous and delicious. This place is heavy and will fill you up so come hungry."

### Loadind reviews into a DataFrame

In [67]:
df = pd.DataFrame(np.array(reviews),columns=["reviews about Dishoom"])

In [71]:
df["reviews about Dishoom"].iloc[0]

"This was a total hit. We made a reservation for lunch which I highly recommend because this place had a line out the door despite not even being open when we arrived. It's so cutely decorated. Tons of nods to the original Cafe Britannia in Bombay and overall seems to serve as a beautiful homage to India/Bombay. The food itself is incredible. Chili cheese toast was authentic. Pau bhaji and chicken were sumptuous and delicious. This place is heavy and will fill you up so come hungry."

In [73]:
#this a function that returns the score (from 1 to 5) about the review given.
def sentiment_score(review):

    tokens = tokenization_tool.encode(review , return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [74]:
sentiment_score(df["reviews about Dishoom"].iloc[0])

5

In [76]:
df["sentiments"] = df["reviews about Dishoom"].apply(lambda x : sentiment_score(x))

In [80]:
df

,reviews about Dishoom,sentiments
0,This was a total hit. We made a reservation fo...,5
1,Dined here on my first day in London. This cam...,5
2,"Went here entirely based on the Yelp reviews, ...",5
3,Dishoom has always been one of my favorites an...,5
4,Have only been here for brunch so that's what ...,3
5,Came around 6:30 on a Thursday and it took aro...,4
6,My 10 year old has declared this the best meal...,5
7,Greeted outside with a warm chai while waiting...,4
8,"I've heard so many good things about Dishoom, ...",4
9,"TLDR: Great atmosphere for a visitor, good foo...",5
